## MODEL RANDOM FOREST

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error  # IMPORTA MAE

In [ ]:
# Carrega de dades 
BASE_PATH = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\Conjunt de dades Preprocessades\Datasets"
file_name = "S&P500_Stock_Price_output.csv"
file_path = os.path.join(BASE_PATH, file_name)

df = pd.read_csv(file_path)

# Preprocessament inicial 
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date').reset_index(drop=True)

# Enginyeria de característiques (només lags de 'Close')
for lag in [1, 2, 3, 5, 10]:
    df[f'Close_lag{lag}'] = df['Close'].shift(lag)

# Definició de X i y 
target_col   = "Close"
feature_cols = [c for c in df.columns if c not in ['Date', target_col]]
X = df[feature_cols].copy()
y = df[target_col].copy()

# Split temporal 70 % train_val / 30 % test
split_index = int(len(df) * 0.7)
X_train_val = X.iloc[:split_index].reset_index(drop=True)
y_train_val = y.iloc[:split_index].reset_index(drop=True)
X_test      = X.iloc[split_index:].reset_index(drop=True)
y_test      = y.iloc[split_index:].reset_index(drop=True)

# Funció d'augmentació (jittering) 
def augment_features(X_orig, y_orig, n_copies=3, noise_level=0.01, random_state=42):
 
    np.random.seed(random_state)
    cont_cols = X_orig.columns.tolist()

    X_list = [X_orig.copy()]
    y_list = [y_orig.copy()]

    for _ in range(n_copies):
        X_aug = X_orig.copy()
        stds = X_orig[cont_cols].std().values  # desviació de cada feature
        noise = np.random.normal(loc=0.0, scale=1.0, size=X_orig[cont_cols].shape)
        noise = noise * (noise_level * stds)  # escalar soroll per feature

        X_aug.loc[:, cont_cols] = X_orig[cont_cols] + noise
        X_list.append(X_aug)
        y_list.append(y_orig.copy())

    X_all = pd.concat(X_list, ignore_index=True)
    y_all = pd.concat(y_list, ignore_index=True)
    return X_all, y_all

# Realitzem augmentació sobre el conjunt de train_val
X_train_aug, y_train_aug = augment_features(
    X_train_val,
    y_train_val,
    n_copies=5,
    noise_level=0.01,
    random_state=42
)

print(f"Mostres train original: {X_train_val.shape[0]}")
print(f"Mostres train augmentat: {X_train_aug.shape[0]}\n")

# Hyperparameter tuning amb RandomizedSearchCV + TimeSeriesSplit sobre X_train_aug
param_dist = {
    'n_estimators':      [int(x) for x in np.linspace(100, 500, num=5)], 
    'max_depth':         [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf':  [1, 2, 4],
    'max_features':      ['auto', 'sqrt', 'log2']
}

tscv = TimeSeriesSplit(n_splits=5)
rf   = RandomForestRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,                # 50 iteracions aleatòries
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

random_search.fit(X_train_aug, y_train_aug)
best_params = random_search.best_params_

print("Millors hiperparàmetres trobats (RandomizedSearchCV amb augmentació):")
for param, val in best_params.items():
    print(f"  {param}: {val}")
print()

# Entrenar model final sobre TOT train_val AUGMENTAT 
final_rf = RandomForestRegressor(**best_params, random_state=42)
final_rf.fit(X_train_aug, y_train_aug)

# Avaluació sobre Test (sense augmentar el test) 
y_pred_test = final_rf.predict(X_test)
rmse_test   = np.sqrt(mean_squared_error(y_test, y_pred_test))  
mae_test    = mean_absolute_error(y_test, y_pred_test)            
r2_test     = r2_score(y_test, y_pred_test)

print("Avaluació final sobre Test:")
print(f"  RMSE Test: {rmse_test:.4f}")
print(f"  MAE  Test: {mae_test:.4f}")   
print(f"  R²   Test: {r2_test:.4f}\n")

# Selecció de variables per importància
importances = final_rf.feature_importances_
feat_imp = pd.DataFrame({
    'feature':    X_train_val.columns,
    'importance': importances
}).sort_values('importance', ascending=False).reset_index(drop=True)

print("Top 10 features per importància:")
print(feat_imp.head(10))

# Eliminar les variables amb importància < llindar
llindar      = 0.01
features_sel = feat_imp[feat_imp['importance'] >= llindar]['feature'].tolist()

print(f"\nVariables seleccionades (importància ≥ {llindar}): {len(features_sel)} de {len(feature_cols)}\n")

X_train_val_red = X_train_val[features_sel]
X_test_red      = X_test[features_sel]

# Entrenem un model reduït (sense augmentar en aquest pas)
final_rf_red = RandomForestRegressor(**best_params, random_state=42)
final_rf_red.fit(X_train_val_red, y_train_val)
y_pred_red = final_rf_red.predict(X_test_red)

rmse_red = np.sqrt(mean_squared_error(y_test, y_pred_red))  
mae_red  = mean_absolute_error(y_test, y_pred_red)               
r2_red   = r2_score(y_test, y_pred_red)

# Mètriques mdoel reduït
print("Model reduït (pilotatge de features):")
print(f"  RMSE Test (reduït): {rmse_red:.4f}")
print(f"  MAE  Test (reduït): {mae_red:.4f}")   
print(f"  R²   Test (reduït): {r2_red:.4f}")


Mostres train original: 894
Mostres train augmentat: 5364

Fitting 5 folds for each of 50 candidates, totalling 250 fits


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:528: FitFailedWarning:


75 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py", line 1382, in wrapper
    estimator._validate_params()
  File "c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.

Millors hiperparàmetres trobats (RandomizedSearchCV amb augmentació):
  n_estimators: 300
  min_samples_split: 5
  min_samples_leaf: 1
  max_features: sqrt
  max_depth: 20

Avaluació final sobre Test:
  RMSE Test: 467763.6603
  MAE  Test: 512.5705
  R²   Test: -0.4566

Top 10 features per importància:
       feature  importance
0          Low    0.216481
1         High    0.207479
2         Open    0.152913
3        EMA_7    0.136083
4   Close_lag1    0.093531
5   Close_lag2    0.082658
6   Close_lag3    0.046920
7       EMA_40    0.032044
8   Close_lag5    0.018680
9  Close_lag10    0.004178

Variables seleccionades (importància ≥ 0.01): 9 de 16

Model reduït (pilotatge de features):
  RMSE Test (reduït): 457607.1621
  MAE  Test (reduït): 505.1915
  R²   Test (reduït): -0.4250


In [ ]:
base_results_folder = r"C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\RANDOM FOREST\resultats_RANDOM_FOREST"
subfolder = "S&P500_Stock_Price_output"
model_folder = os.path.join(base_results_folder, subfolder)
dataset_name = "S&P500"
# Ens assegurem que la carpeta existeixi
os.makedirs(model_folder, exist_ok=True)

# Construir un DataFrame amb les mètriques
metrics_red = pd.DataFrame({
    "Model": ["RandomForest_Reducit"],
    "RMSE_test": [rmse_red],
    "MAE_test":  [mae_red],
    "R2_test":   [r2_red]
})

# Escriure el CSV amb les mètriques
metrics_csv_path = os.path.join(model_folder, f"{dataset_name}_reduced_metrics.csv")
metrics_red.to_csv(metrics_csv_path, index=False)

print(f"  ✓ Mètriques del model reduït guardades a: {metrics_csv_path}")

  ✓ Mètriques del model reduït guardades a: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\RANDOM FOREST\resultats_RANDOM_FOREST\S&P500_Stock_Price_output\S&P500_reduced_metrics.csv


In [ ]:
import os
from collections import deque
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestRegressor

# Gràfica Real vs Predit (Test) amb el model reduït
# Recuperar les dates de test a partir del DataFrame original:
dates_test = df['Date'].iloc[split_index:].reset_index(drop=True)

# Construir les sèries "real" i "predit" per graficar
y_true_red = y_test.reset_index(drop=True)               
y_pred_red  = pd.Series(y_pred_red).reset_index(drop=True)  

# Plotly: Real vs Predit
fig_test = go.Figure()
fig_test.add_trace(go.Scatter(
    x=dates_test,
    y=y_true_red,
    mode='lines',
    name='Real (Close)',
    line=dict(color='blue')
))
fig_test.add_trace(go.Scatter(
    x=dates_test,
    y=y_pred_red,
    mode='lines',
    name='Predit (RF reduït)',
    line=dict(color='red', dash='dash')
))
fig_test.update_layout(
    title="S&P500 – Real vs Predit (Test) [Model reduït]",
    xaxis_title='Data',
    yaxis_title='Preu Close (USD)',
    template='plotly_dark',
    xaxis_rangeslider_visible=True
)

# Desar la gràfica en HTML
test_plot_path = os.path.join(model_folder, f"{dataset_name}_test_reduced_plot.html")
fig_test.write_html(test_plot_path)
print(f"  ✓ Gràfica Test (model reduït) desada a: {test_plot_path}")


# Preparació per a la predicció autoregressiva de 10 dies
# Seleccionar només les columnes de lags a features_sel
lag_features = [f for f in features_sel if f.startswith("Close_lag")]

if len(lag_features) == 0:
    raise ValueError(
        "No hi ha columnes 'Close_lag' a features_sel. "
        "El model reduït no fa servir cap lag de 'Close', per tant no és possible fer predicció autoregressiva."
    )

# Si features_sel incloïa altres variables (p. ex. 'Open', 'High', etc.),
#        reentrenem un model només amb lag_features per les prediccions futures:
if len(lag_features) < len(features_sel):
    print("⚠️  S'ha detectat que features_sel inclou variables diferents de lags de 'Close'.")
    print("    Reentrenant un nou RandomForestRegressor només amb:", lag_features)
    final_rf_lag = RandomForestRegressor(**best_params, random_state=42)
    final_rf_lag.fit(X_train_val[lag_features], y_train_val)
    modelo_para_futuro = final_rf_lag
else:
    # Si només hi havia lags, fem servir final_rf_red directament:
    modelo_para_futuro = final_rf_red

# Determinar el màxim lag (p. ex. Close_lag10 → max_lag = 10)
max_lag = max(int(f.split("Close_lag")[1]) for f in lag_features)

# Crear deque amb els últims 'max_lag' valors reals de df['Close']
last_closes = deque(df['Close'].iloc[-max_lag:].values, maxlen=max_lag)

# Generar els propers 10 dies laborables
future_dates = pd.bdate_range(
    start=df['Date'].iloc[-1] + pd.Timedelta(days=1),
    periods=10
)

future_preds = []

# Bucle dia a dia per predir de manera autoregressiva
for fecha in future_dates:
    # Construir un dict X_new amb tots els lag_features
    X_new = {}
    for f in lag_features:
        lag_num = int(f.split("Close_lag")[1])
        X_new[f] = last_closes[-lag_num]

    # Convertir a DataFrame d’una sola fila
    X_new_df = pd.DataFrame([X_new])

    # Predir el proper 'Close'
    y_pred_fut = modelo_para_futuro.predict(X_new_df)[0]
    future_preds.append(y_pred_fut)

    # Afegir la predicció a last_closes
    last_closes.append(y_pred_fut)

# Desar prediccions futures en un CSV
df_fut_pred = pd.DataFrame({
    "Date": future_dates,
    "Predicted_Close": future_preds
})
future_csv_path = os.path.join(model_folder, f"{dataset_name}_future_10days_reduced.csv")
df_fut_pred.to_csv(future_csv_path, index=False)
print(f"  ✓ Prediccions futures (model reduït) desades a: {future_csv_path}")


# Gràfica històric + prediccions futures

fig_future = go.Figure()
# 13.1) Històric real de 'Close'
fig_future.add_trace(go.Scatter(
    x=df['Date'],
    y=df['Close'],
    mode='lines',
    name='Històric Close',
    line=dict(color='lightblue')
))
# 13.2) Sèrie de prediccions futures
fig_future.add_trace(go.Scatter(
    x=future_dates,
    y=np.array(future_preds),
    mode='lines+markers',
    name='Predicció futura (10 dies)',
    line=dict(color='orange', dash='dash'),
    marker=dict(size=6)
))
fig_future.update_layout(
    title="S&P500 – Predicció Propers 10 Dies (Model reduït)",
    xaxis_title='Data',
    yaxis_title='Preu Close (USD)',
    template='plotly_dark',
    xaxis_rangeslider_visible=True
)

future_plot_path = os.path.join(model_folder, f"{dataset_name}_future_reduced_plot.html")
fig_future.write_html(future_plot_path)
print(f"  ✓ Gràfica futura (model reduït) desada a: {future_plot_path}")


  ✓ Gráfica Test (modelo reducido) guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\RANDOM FOREST\resultats_RANDOM_FOREST\Amazon_Stock_Price_output\Amazon_test_reduced_plot.html
⚠️  Detectado que features_sel incluye variables distintas a lags de 'Close'.
    Reentrenando un nuevo RandomForestRegressor usando únicamente: ['Close_lag1', 'Close_lag2', 'Close_lag3', 'Close_lag5', 'Close_lag10']
  ✓ Predicciones futuras (modelo reducido) guardadas en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\RANDOM FOREST\resultats_RANDOM_FOREST\Amazon_Stock_Price_output\Amazon_future_10days_reduced.csv
  ✓ Gráfica futura (modelo reducido) guardada en: C:\Users\jesus\Desktop\TFG\GitHUb\TFG_PredictStock\RANDOM FOREST\resultats_RANDOM_FOREST\Amazon_Stock_Price_output\Amazon_future_reduced_plot.html
